In [16]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import chess
import numpy as np
from math import ceil

In [5]:
data=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\clean_games.csv')
data.Scores=data.Scores.str.replace(' ','').str.replace('[','').str.replace(']','').str.replace(',',' ')
data.Scores=data.Scores.apply(lambda x: list(map(int,x.split())))
data.Scores=data.Scores.apply(lambda x: list(map(int,x)))
data.Moves=data.Moves.str.replace('"','').str.replace("'","").str.replace('[','').str.replace(']','').str.replace(',',' ')
data.Moves=data.Moves.apply(lambda x: x.split())
data.head()

,Result,Moves,Scores,NumMoves,MeanScore,ModeScore,FinalScore
0,1/2-1/2,"[Nf3, Nf6, c4, c5, b3, g6, Bb2, Bg7, e3, O-O, ...","[18, 17, 12, 8, -5, 12, 3, -2, 22, 21, 20, 13,...",19,23.947368,3,54
1,0-1,"[e4, d5, exd5, Nf6, d4, Nxd5, Nf3, g6, Be2, Bg...","[26, 51, 68, 57, 65, 77, 48, 93, 61, 63, 63, 5...",53,-692.198113,18,-11544
2,1-0,"[c4, Nf6, Nc3, d6, d4, e5, Nf3, Nbd7, Bg5, Be7...","[2, 21, 5, 53, 35, 45, 37, 54, 10, 22, 8, 48, ...",39,49.246753,31,196
3,1-0,"[e4, c5, Nf3, d6, b4, Nf6, bxc5, Nxe4, cxd6, Q...","[26, 64, 35, 53, 18, 20, 18, 20, 10, 49, 60, 9...",25,306.040816,18,875
4,1/2-1/2,"[Nf3, d5, d4, Nf6, c4, e6, Nc3, Be7, Bg5, h6, ...","[18, 29, 10, 22, 20, 18, 19, 28, 24, 18, 17, 1...",29,30.827586,18,-11


In [7]:
elos=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\clean_elos.csv')
elos.head()

,WhiteElo,BlackElo,MeanElos,DiffElos,SumElos
0,2354,2411,2382,57,4765
1,1915,1999,1957,84,3914
2,2446,2191,2318,255,4637
3,2168,2075,2121,93,4243
4,2437,2254,2345,183,4691


In [8]:
features=data.copy()
features.Result=features.Result.apply(lambda x: 0.5 if x=='1/2-1/2' else x)
features.Result=features.Result.apply(lambda x: 1 if x=='1-0' else x)
features.Result=features.Result.apply(lambda x: 0 if x=='0-1' else x)
features=features.drop(columns=['Moves','Scores'])
features.head()

,Result,NumMoves,MeanScore,ModeScore,FinalScore
0,0.5,19,23.947368,3,54
1,0.0,53,-692.198113,18,-11544
2,1.0,39,49.246753,31,196
3,1.0,25,306.040816,18,875
4,0.5,29,30.827586,18,-11


#### Общие оценки

In [9]:
features['MinScore']=data.Scores.apply(lambda x: min(x))
features['MaxScore']=data.Scores.apply(lambda x: max(x))
features['MedianScore']=data.Scores.apply(lambda x: np.median(x))
features['StdDevScore']=data.Scores.apply(lambda x: np.std(x))
features.head()

,Result,NumMoves,MeanScore,ModeScore,FinalScore,MinScore,MaxScore,MedianScore,StdDevScore
0,0.5,19,23.947368,3,54,-26,73,18.0,26.431587
1,0.0,53,-692.198113,18,-11544,-11544,93,-48.0,2286.131735
2,1.0,39,49.246753,31,196,-30,198,37.0,51.525093
3,1.0,25,306.040816,18,875,10,917,142.0,313.263595
4,0.5,29,30.827586,18,-11,-15,94,29.0,24.845827


#### Общая разница в оценках

In [10]:
def delta(x,func):
    diffs=[]
    for i in range(len(x)-1):
        diffs.append((x[i+1]-x[i]))
    if func=='mean':
        return np.mean(diffs)
    elif func=='min':
        return min(diffs)
    elif func=='max':
        return max(diffs)
    elif func=='median':
        return np.median(diffs)
    elif func=='std':
        return np.std(diffs)

features['MeanDeltaScore']=data.Scores.apply(delta,args=('mean',))
features['MinDeltaScore']=data.Scores.apply(delta,args=('min',))
features['MaxDeltaScore']=data.Scores.apply(delta,args=('max',))
features['MedianDeltaScore']=data.Scores.apply(delta,args=('median',))
features['StdDevDeltaScore']=data.Scores.apply(delta,args=('std',))
features.head()

,Result,NumMoves,MeanScore,ModeScore,FinalScore,MinScore,MaxScore,MedianScore,StdDevScore,MeanDeltaScore,MinDeltaScore,MaxDeltaScore,MedianDeltaScore,StdDevDeltaScore
0,0.5,19,23.947368,3,54,-26,73,18.0,26.431587,0.972973,-27,44,-4.0,16.203914
1,0.0,53,-692.198113,18,-11544,-11544,93,-48.0,2286.131735,-110.190476,-9578,5009,-4.0,1090.554329
2,1.0,39,49.246753,31,196,-30,198,37.0,51.525093,2.552632,-69,85,-1.0,25.355055
3,1.0,25,306.040816,18,875,10,917,142.0,313.263595,17.687500,-87,309,5.0,67.765575
4,0.5,29,30.827586,18,-11,-15,94,29.0,24.845827,-0.508772,-57,29,1.0,15.034607


#### Признаки только по отдельным цветам

In [11]:
data['WhiteMoves']=data.Moves.apply(lambda x: x[::2])
data['WhiteScores']=data.Scores.apply(lambda x: x[::2])
data['BlackMoves']=data.Moves.apply(lambda x: x[1::2])
data['BlackScores']=data.Scores.apply(lambda x: x[1::2])
data=data.drop(columns=['NumMoves','MeanScore','ModeScore','FinalScore'])
data.head()

,Result,Moves,Scores,WhiteMoves,WhiteScores,BlackMoves,BlackScores
0,1/2-1/2,"[Nf3, Nf6, c4, c5, b3, g6, Bb2, Bg7, e3, O-O, ...","[18, 17, 12, 8, -5, 12, 3, -2, 22, 21, 20, 13,...","[Nf3, c4, b3, Bb2, e3, Be2, O-O, Nc3, Qc2, Rac...","[18, 12, -5, 3, 22, 20, 8, 11, -6, 1, -21, -26...","[Nf6, c5, g6, Bg7, O-O, b6, Bb7, Nc6, Rc8, d5,...","[17, 8, 12, -2, 21, 13, 21, 3, 5, -10, -1, 18,..."
1,0-1,"[e4, d5, exd5, Nf6, d4, Nxd5, Nf3, g6, Be2, Bg...","[26, 51, 68, 57, 65, 77, 48, 93, 61, 63, 63, 5...","[e4, exd5, d4, Nf3, Be2, c4, Nc3, O-O, Be3, d5...","[26, 68, 65, 48, 61, 63, 53, 69, 30, -2, -11, ...","[d5, Nf6, Nxd5, g6, Bg7, Nb6, O-O, Nc6, Bg4, B...","[51, 57, 77, 93, 63, 58, 46, 29, 27, 12, 0, 24..."
2,1-0,"[c4, Nf6, Nc3, d6, d4, e5, Nf3, Nbd7, Bg5, Be7...","[2, 21, 5, 53, 35, 45, 37, 54, 10, 22, 8, 48, ...","[c4, Nc3, d4, Nf3, Bg5, e3, Qc2, Bd3, h4, dxe5...","[2, 5, 35, 37, 10, 8, 30, 13, -12, 13, -22, -1...","[Nf6, d6, e5, Nbd7, Be7, c6, O-O, h6, b5, dxe5...","[21, 53, 45, 54, 22, 48, 17, 35, 31, 31, 20, -..."
3,1-0,"[e4, c5, Nf3, d6, b4, Nf6, bxc5, Nxe4, cxd6, Q...","[26, 64, 35, 53, 18, 20, 18, 20, 10, 49, 60, 9...","[e4, Nf3, b4, bxc5, cxd6, d4, Bd3, O-O, Be3, B...","[26, 35, 18, 18, 10, 60, 91, 83, 58, 98, 146, ...","[c5, d6, Nf6, Nxe4, Qb6, Bg4, Nxd6, Nc6, Nb4, ...","[64, 53, 20, 20, 49, 95, 82, 93, 59, 142, 193,..."
4,1/2-1/2,"[Nf3, d5, d4, Nf6, c4, e6, Nc3, Be7, Bg5, h6, ...","[18, 29, 10, 22, 20, 18, 19, 28, 24, 18, 17, 1...","[Nf3, d4, c4, Nc3, Bg5, Bh4, e3, Bxe7, Qc2, Qx...","[18, 10, 20, 19, 24, 17, 18, 48, 51, 44, 41, 2...","[d5, Nf6, e6, Be7, h6, O-O, Ne4, Qxe7, Nxc3, d...","[29, 22, 18, 28, 18, 16, 38, 37, 53, 48, 16, 1..."


In [12]:
features['MinWhiteScore']=data.WhiteScores.apply(lambda x: min(x))
features['MaxWhiteScore']=data.WhiteScores.apply(lambda x: max(x))
features['MedianWhiteScore']=data.WhiteScores.apply(lambda x: np.median(x))
features['StdDevWhiteScore']=data.WhiteScores.apply(lambda x: np.std(x))
features['MeanWhiteScore']=data.WhiteScores.apply(lambda x: np.mean(x))
features['ModeWhiteScore']=data.WhiteScores.apply(lambda x: max(set(x), key=x.count))
features['FinalWhiteScore']=data.WhiteScores.apply(lambda x: x[-1])

features['MinBlackScore']=data.BlackScores.apply(lambda x: min(x))
features['MaxBlackScore']=data.BlackScores.apply(lambda x: max(x))
features['MedianBlackScore']=data.BlackScores.apply(lambda x: np.median(x))
features['StdDevBlackScore']=data.BlackScores.apply(lambda x: np.std(x))
features['MeanBlackScore']=data.BlackScores.apply(lambda x: np.mean(x))
features['ModeBlackScore']=data.BlackScores.apply(lambda x: max(set(x), key=x.count))
features['FinalBlackScore']=data.BlackScores.apply(lambda x: x[-1])

features.shape

(21792, 28)

#### Разница в оценках по цветам

In [13]:
features['MeanWhiteDeltaScore']=data.WhiteScores.apply(delta,args=('mean',))
features['MinWhiteDeltaScore']=data.WhiteScores.apply(delta,args=('min',))
features['MaxWhiteDeltaScore']=data.WhiteScores.apply(delta,args=('max',))
features['MedianWhiteDeltaScore']=data.WhiteScores.apply(delta,args=('median',))
features['StdDevWhiteDeltaScore']=data.WhiteScores.apply(delta,args=('std',))

features['MeanBlackDeltaScore']=data.BlackScores.apply(delta,args=('mean',))
features['MinBlackDeltaScore']=data.BlackScores.apply(delta,args=('min',))
features['MaxBlackDeltaScore']=data.BlackScores.apply(delta,args=('max',))
features['MedianBlackDeltaScore']=data.BlackScores.apply(delta,args=('median',))
features['StdDevBlackDeltaScore']=data.BlackScores.apply(delta,args=('std',))

features.shape

(21792, 38)

#### Индекс первого хода, когда белые или черные получили преимущество (>100 cp)
+победили ли они после получения преимущества (если не было преимущества - то сверяем с ничьей)

In [37]:
def find_advantage(scores):
    for i in range(len(scores)):
        if abs(scores[i]) >=100:
            return i
    return 0

def is_realised_advantage(ind,result):
    if ind==0 and result==0.5:
        return 1
    elif ind!=0 and ind %2!=0 and result==1:
        return 1
    elif ind!=0 and ind %2==0 and result==0:
        return 1
    else:
        return 0        
    
features['FirstAdvantageInd']=data.Scores.apply(find_advantage)
features['IsAdvantageRealised']=features[['FirstAdvantageInd','Result']].apply(lambda x: is_realised_advantage(*x),axis=1)
features.head(10)

,Result,NumMoves,MeanScore,ModeScore,FinalScore,MinScore,MaxScore,MedianScore,StdDevScore,MeanDeltaScore,...,MedianWhiteDeltaScore,StdDevWhiteDeltaScore,MeanBlackDeltaScore,MinBlackDeltaScore,MaxBlackDeltaScore,MedianBlackDeltaScore,StdDevBlackDeltaScore,FirstAdvantage,FirstAdvantageInd,IsAdvantageRealised
0,0.5,19,23.947368,3,54,-26,73,18.0,26.431587,0.972973,...,0.5,22.114531,2.055556,-18,30,-1.5,14.905153,0,0,1
1,0.0,53,-692.198113,18,-11544,-11544,93,-48.0,2286.131735,-110.190476,...,-9.0,767.218965,-222.980769,-9685,3830,-5.0,1518.048956,29,58,1
2,1.0,39,49.246753,31,196,-30,198,37.0,51.525093,2.552632,...,2.0,25.605087,4.351351,-50,68,3.0,23.903201,33,65,1
3,1.0,25,306.040816,18,875,10,917,142.0,313.263595,17.687500,...,16.0,78.221274,34.956522,-64,286,16.0,84.087676,10,19,1
4,0.5,29,30.827586,18,-11,-15,94,29.0,24.845827,-0.508772,...,1.0,21.645421,-1.428571,-53,34,-1.5,18.072134,0,0,1
5,1.0,38,237.440000,32,898,-4,6395,104.0,744.029556,11.851351,...,4.0,65.416695,177.333333,-65,5539,9.0,909.317326,14,27,1
6,0.5,64,-1081.496063,0,0,-10293,64,-100.0,2132.508389,-0.206349,...,-2.0,848.578524,-1.032258,-5372,10293,-2.0,1832.253495,17,34,0
7,1.0,46,106.923077,37,410,-3,445,83.0,93.119943,4.266667,...,3.0,28.120850,6.068182,-149,251,0.0,52.622324,31,61,1
8,1.0,38,253.746667,39,1240,15,1581,65.0,345.365356,16.405405,...,10.0,74.848618,41.305556,-35,378,11.5,92.997258,22,43,1
9,0.0,50,-239.680000,6,-682,-683,63,-298.0,256.674225,-7.151515,...,-4.0,34.832018,-15.142857,-154,45,-5.0,35.913729,23,45,0


In [39]:
features.to_csv(r'C:\Users\Asus\PredictingRatings\data\features.csv', sep=',',index=False)